In [ ]:
!pip install ieee754

In [ ]:
from ieee754 import single
import random
import numpy as np
import plotly
import plotly.graph_objs as go

def fBIN_to_fDEC(a):
    n=2
    k = 0
    a=list(a)
    for i in range(len(a)):
        k += int(a[i]) * float(n**(-1*(i+1)))
    return k

def bin_to_dec(a):
     n=2
     k = 0
     for i in range(len(a)):
         k += int(a[len(a)-i-1]) * int(n**i)
     return k

def bin2dec(num):
  # разбиенеи на составляющие
  sign=int(num[0])
  bin_exponent= num[1:9]
  bin_mantissa= num[9:]

  exponent=bin_to_dec(bin_exponent)
  mantissa=1+fBIN_to_fDEC(bin_mantissa)

  return round((-1)**sign * mantissa * 2**(exponent-127), 6)


def dec2bin(num): return ''.join(str(single(num)).split())

def chunks(mas, chunk_size):#разделение спсика на группы(не важно соклько) определенного размера(важно какого размера)
    n=chunk_size
    n = max(1, n)
    return [mas[i:i+n] for i in range(0, len(mas), n)]

def split(mas, part_size):#разделение спсика на определенные группы(важно соклько групп) примерно равного размера размера(не важно каого размера)
  n=part_size
  k, m = divmod(len(mas), n)
  return [mas[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)]

x=[
[0,0,0,0],
[0,0,0,1],
[0,0,1,0],
[0,0,1,1],
[0,1,0,0],
[0,1,0,1],
[0,1,1,0],
[0,1,1,1],
[1,0,0,0],
[1,0,0,1],
[1,0,1,0],
[1,0,1,1],
[1,1,0,0],
[1,1,0,1],
[1,1,1,0],
[1,1,1,1]]

target= [1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1]
# terget=[1,1,1,0,1,0,0,0,1,0,0,0,1,1,0,1]
lt=len(target)

def neuron (w, x):
  net = x[0]*w[1] + x[1]*w[2] + x[2]*w[3] + x[3]*w[4] + w[0] #сумматор
  out = 1 if net>=0 else 0 #активатор
  return out

def mutation(bin_chromosome):
  bin_chromosome=list(bin_chromosome)
  r=random.randint(0, len(bin_chromosome)-1)
  i=int(bin_chromosome[r])
  bin_chromosome[r] =  ('1', '0')[i]
  return ''.join(bin_chromosome)

N=128 #число особей(размер популяции) population_size
w=5 #число храмосом(весов, признаков) у особи
gen=20 #поколений

min_result=[]

# создание начальной популяции
population = []
for _ in range(N):
  individ = []
  for _ in range(w):
    chromosome = round(random.uniform(-1, 1), 3)
    individ.append(chromosome)
  population.append(individ)

strong_individual=0
# процесс эвалюции
for g in range(gen):
  answer_population=[]
  for individ in population:
    answer_individ =  []
    for i in x:
      answer_individ.append(neuron(individ, i))
    answer_population.append(answer_individ)

  correct_answer_population=[]
  for answer in answer_population:
    sum = 0 #число верных ответов особи
    for Y, y in zip(target, answer):
      if Y == y : sum+=1
    correct_answer_population.append(sum)
  max_ans=max(correct_answer_population)
  min_result.append(lt-max_ans)
  strong_individual=correct_answer_population.index(max_ans)

  population = [[individ,correct_answer] for individ, correct_answer in zip(population, correct_answer_population)]#сопаставление особи и его количества верных ответов
  population.sort(key = lambda x: x[1])#отсортировали по количеству вверных ответов тем самым получили ранги
  selected=[population[-1][0], population[-2][0]]#элитивизм (прошли две самые сильные)
  stayed=population[:-2]

  # создаем пространство для выбора особей методом ранка
  space=[]
  for individ in stayed:
    space.extend([individ[0]]*individ[1])

  # добавляем случайным образом из пространства тех кто будет участвовать в скрещивании
  selected.extend(random.sample(space, int(N/2)-2))

  # клонирование до размера популяции
  selected.extend(selected)

  # перемешивание и разбиение на пары
  random.shuffle(selected)
  selected=chunks(selected, 2)

  # скрещивание
  new_population=[]
  for pair_individ in selected:
    descendent1=[]
    descendent2=[]
    for chromosome1, chromosome2 in zip(pair_individ[0], pair_individ[1]):
      # делим храмомосому на 3 части
      chromosome1 = split(dec2bin(chromosome1), 3)
      chromosome2 = split(dec2bin(chromosome2), 3)
      #меняем средню часть между храмосомами
      chromosome1[1], chromosome2[1] = chromosome2[1], chromosome1[1]

      # мутация хромосом
      chromosome1=mutation(''.join(chromosome1))
      chromosome2=mutation(''.join(chromosome2))

      descendent1.append(round(bin2dec(chromosome1),4))
      descendent2.append(round(bin2dec(chromosome2),4))

    new_population.append(descendent1)
    new_population.append(descendent2)
  population=new_population

print('Лучшее решение:', population[strong_individual])

# строим графики
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(gen)), y=min_result, mode='lines+markers', name = ""))
fig.update_traces(hoverinfo="all", hovertemplate="Поколение: %{x}<br>Результат: %{y}")
fig.update_layout(xaxis_title="№ поколения",yaxis_title="Кол-во ошибок")

# выводим графики в браузер
fig.show()

Лучшее решение: [-32.0039, 1.0161419250455611e+31, 3.6893488147419103e+19, 1.2682459903621446e+38, 0.0]
